In [76]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "Give me 10 words on {topic}"
)
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=512,
)  
output_parser = StrOutputParser()

In [77]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=output_parser
)

# and run
out = chain.run(topic="Artificial Intelligence")
print(out)

Innovative, intelligent, futuristic, automated, self-learning, efficient, advanced, transformative, complex, powerful.


In [78]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What was your last answer?")

In [79]:
lcel_chain = chat_template | model | output_parser

out = lcel_chain.invoke({"name": "Bob", "user_input": "What was your last answer?"})

In [80]:
print(out)

My last answer was "I'm doing well, thanks!"


In [81]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

vecstore_a = DocArrayInMemorySearch.from_texts(
    ["half the info will be here", "James' birthday is the 7th December"],
    embedding=embedding
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    ["and half here", "James was born in 1994"],
    embedding=embedding
)

In [82]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

prompt_str = """Answer the question below using the context:

Context: {context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {"context": retriever_a, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | model | output_parser

In [83]:
chain.invoke("When was James born?")

'James was born on the 7th of December.'

In [94]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

prompt_str = """Answer the question below ONLY using the context_a and context_b below:

context_a: {context_a}
\n
context_b: {context_b}
\n
Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {"context_a": retriever_a, "context_b":retriever_b, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | model | output_parser

In [96]:
chain.invoke( "what date with year exactly was James born?")

'James was born on the 7th December in 1994.'

In [89]:
from langchain_core.runnables import RunnableLambda

def add_five(x):
    return x+5

def multiply_by_two(x):
    return x*2

# wrap the functions with RunnableLambda
add_five = RunnableLambda(add_five)
multiply_by_two = RunnableLambda(multiply_by_two)

In [92]:
chain = add_five | multiply_by_two
chain.invoke(1)

12

In [103]:
import pandas as pd

# Create a dictionary with sample data
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, 30, 35, 40, 45],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

df


,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago
3,David,40,Houston
4,Eva,45,Phoenix


In [117]:
def execute_function(function_description):
    exec(func)
    function_df(df)

In [121]:
import pandas as pd

# Create a dictionary with sample data
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, 30, 35, 40, 45],
    'City': ['New York, Bn2 4ll', 'Los Angeles, SE10 0UH', 'Chicago', 'Houston', 'Phoenix']
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)


p1 = ChatPromptTemplate.from_template("""Give me a concise python function to perform {query} on a pandas dataframe. 
                                      Just give the function without
                                      comments or anything else. Call the function function_df. Just give the function without any imports""")
c1 = p1 | model| output_parser
display(df)
func = c1.invoke({"query":"In the City column, it contains some postcodes, but in different formats, I want to make the format of all postcodes the same to be AB12 2CD for example."})

,Name,Age,City
0,Alice,25,"New York, Bn2 4ll"
1,Bob,30,"Los Angeles, SE10 0UH"
2,Charlie,35,Chicago
3,David,40,Houston
4,Eva,45,Phoenix


In [122]:
func

"import pandas as pd\n\ndef function_df(df):\n    df['City'] = df['City'].str.replace(r'(\\w{2})(\\d{1,2})(\\d{1})(\\w{2})', r'\\1\\2 \\3\\4')\n    return df"

In [ ]:
exec(func)
    function_df(df)

In [119]:
df

,Name,Age,City,y
0,Alice,25,"New York, Bn2 4ll",35.0
1,Bob,30,"Los Angeles, SE10 0UH",35.0
2,Charlie,35,Chicago,35.0
3,David,40,Houston,35.0
4,Eva,45,Phoenix,35.0


In [106]:
func

"def function_df(df):\n    df['year'] = 2024\n    return df"

In [ ]:

exec(function_description)

prompt = ChatPromptTemplate.from_template("Give me a concise python function to perform {query} on a pandas dataframe.")

chain = prompt | RunnableLambda(execute_function)

chain.invoke()


In [97]:
# String description of the function
function_description = """
def my_function(x):
    return x * x
"""


# Now you can use the function
result = my_function(5)
print(result)  # Output: 25


NameError: name 'my_function' is not defined